In [94]:
import json
import pandas as pd
from datetime import datetime

def read_json_file(file_content):
    data = []
    for line in file_content.split('\n'):
        try:
            if line.strip():
                data.append(json.loads(line))
        except json.JSONDecodeError:
            continue
    return data

def create_user_table(users_data):
    users = []
    for user in users_data:
        users.append({
            '_id': user['_id']['$oid'],
            'active': user.get('active', None),
            'createDate': pd.to_datetime(user['createdDate']['$date'], unit='ms') if 'createdDate' in user else None,
            'lastLogin': pd.to_datetime(user['lastLogin']['$date'], unit='ms') if 'lastLogin' in user else None,
            'role': user.get('role', None),
            'signUpSource': user.get('signUpSource', None),
            'state': user.get('state', None)
        })
    return pd.DataFrame(users)

def create_brand_table(brands_data):
    brands = []
    for brand in brands_data:
        brand_dict = {
            '_id': brand['_id']['$oid'],
            'name': brand.get('name', None),
            'topBrand': brand.get('topBrand', None),
            'brandCode': brand.get('brandCode', None)
        }
        brands.append(brand_dict)
    return pd.DataFrame(brands)

def create_product_table(brands_data):
    products = []
    for brand in brands_data:
        product_dict = {
            '_id': brand['_id']['$oid'],
            'barcode': brand.get('barcode', None),
            'category': brand.get('category', None),
            'categoryCode': brand.get('categoryCode', None),
            'cpg_id': brand['cpg']['$id']['$oid'] if '$id' in brand['cpg'] else None,
            'brand_id': brand['_id']['$oid'],
            'description': brand.get('description', None),
            'competitiveProduct': brand.get('competitiveProduct', None)
        }
        products.append(product_dict)
    return pd.DataFrame(products)

def create_cpg_table(brands_data):
    cpgs = set()  # Using set to avoid duplicates
    for brand in brands_data:
        if '$id' in brand['cpg']:
            cpgs.add((
                brand['cpg']['$id']['$oid'],
                brand['cpg'].get('$ref', None)
            ))
    return pd.DataFrame(list(cpgs), columns=['_id', 'ref'])

def create_rewards_table(receipts_data):
    rewards = []
    for receipt in receipts_data:
        if 'rewardsReceiptItemList' in receipt:
            for item in receipt['rewardsReceiptItemList']:
                if 'rewardsGroup' in item or 'rewardsProductPartnerId' in item:
                    reward_dict = {
                        'pointsPayerId': item.get('pointsPayerId', None),
                        'rewardsGroup': item.get('rewardsGroup', None),
                        'rewardsProductPartnerId': item.get('rewardsProductPartnerId', None),
                        'competitorRewardsGroup': item.get('competitorRewardsGroup', None)
                    }
                    rewards.append(reward_dict)
    return pd.DataFrame(rewards).drop_duplicates()

def create_original_metabrite_table(receipts_data):
    metabrite_data = []
    for receipt in receipts_data:
        if 'rewardsReceiptItemList' in receipt:
            for item in receipt['rewardsReceiptItemList']:
                if any(key.startswith('originalMetaBrite') for key in item.keys()):
                    meta_dict = {
                        '_id': receipt['_id']['$oid'] + '_' + item.get('partnerItemId', ''),
                        'originalMetaBriteBarcode': item.get('originalMetaBriteBarcode', None),
                        'originalMetaBriteDescription': item.get('originalMetaBriteDescription', None),
                        'metabriteCampaignId': item.get('metabriteCampaignId', None)
                    }
                    metabrite_data.append(meta_dict)
    return pd.DataFrame(metabrite_data)

def create_receipt_and_items_tables(receipts_data):
    receipts = []
    receipt_items = []

    for receipt in receipts_data:
        # Main receipt data
        receipt_dict = {
            '_id': receipt['_id']['$oid'],
            'bonusPointsEarned': receipt.get('bonusPointsEarned', None),
            'bonusPointsEarnedReason': receipt.get('bonusPointsEarnedReason', None),
            'createDate': pd.to_datetime(receipt['createDate']['$date'], unit='ms') if 'createDate' in receipt else None,
            'dateScanned': pd.to_datetime(receipt['dateScanned']['$date'], unit='ms') if 'dateScanned' in receipt else None,
            'finishedDate': pd.to_datetime(receipt['finishedDate']['$date'], unit='ms') if 'finishedDate' in receipt else None,
            'modifyDate': pd.to_datetime(receipt['modifyDate']['$date'], unit='ms') if 'modifyDate' in receipt else None,
            'pointsAwardedDate': pd.to_datetime(receipt['pointsAwardedDate']['$date'], unit='ms') if 'pointsAwardedDate' in receipt else None,
            'pointsEarned': receipt.get('pointsEarned', None),
            'purchaseDate': pd.to_datetime(receipt['purchaseDate']['$date'], unit='ms') if 'purchaseDate' in receipt else None,
            'purchasedItemCount': receipt.get('purchasedItemCount', None),
            'rewardsReceiptStatus': receipt.get('rewardsReceiptStatus', None),
            'totalSpent': receipt.get('totalSpent', None),
            'userId': receipt.get('userId', None)
        }
        receipts.append(receipt_dict)

        # Receipt items data
        if 'rewardsReceiptItemList' in receipt:
            for item in receipt['rewardsReceiptItemList']:
                item_dict = {
                    '_id': receipt['_id']['$oid'] + '_' + item.get('partnerItemId', ''),
                    'receipt_id': receipt['_id']['$oid'],
                    'barcode': item.get('barcode', None),
                    'description': item.get('description', None),
                    'finalPrice': item.get('finalPrice', None),
                    'itemPrice': item.get('itemPrice', None),
                    'needsFetchReview': item.get('needsFetchReview', None),
                    'partnerItemId': item.get('partnerItemId', None),
                    'preventTargetGapPoints': item.get('preventTargetGapPoints', None),
                    'quantityPurchased': item.get('quantityPurchased', None),
                    'userFlaggedBarcode': item.get('userFlaggedBarcode', None),
                    'userFlaggedNewItem': item.get('userFlaggedNewItem', None),
                    'userFlaggedPrice': item.get('userFlaggedPrice', None),
                    'userFlaggedQuantity': item.get('userFlaggedQuantity', None),
                    'needsFetchReviewReason': item.get('needsFetchReviewReason', None),
                    'pointsNotAwardedReason': item.get('pointsNotAwardedReason', None),
                    'pointsPayerId': item.get('pointsPayerId', None),
                    'rewardsGroup': item.get('rewardsGroup', None),
                    'rewardsProductPartnerId': item.get('rewardsProductPartnerId', None),
                    'userFlaggedDescription': item.get('userFlaggedDescription', None),
                    'originalReceiptItemText': item.get('originalReceiptItemText', None),
                    'pointsEarned': item.get('pointsEarned', None),
                    'targetPrice': item.get('targetPrice', None),
                    'competitiveProduct': item.get('competitiveProduct', None),
                    'originalMetaBrite_id': receipt['_id']['$oid'] + '_' + item.get('partnerItemId', '')
                }
                receipt_items.append(item_dict)

    return pd.DataFrame(receipts), pd.DataFrame(receipt_items)

def process_data(receipts_content, brands_content, users_content):
    # Parse JSON data
    receipts_data = read_json_file(receipts_content)
    brands_data = read_json_file(brands_content)
    users_data = read_json_file(users_content)

    # Create all tables
    users_df = create_user_table(users_data)
    brands_df = create_brand_table(brands_data)
    products_df = create_product_table(brands_data)
    cpg_df = create_cpg_table(brands_data)
    rewards_df = create_rewards_table(receipts_data)
    original_metabrite_df = create_original_metabrite_table(receipts_data)
    receipts_df, receipt_items_df = create_receipt_and_items_tables(receipts_data)

    return {
        'users': users_df,
        'brands': brands_df,
        'products': products_df,
        'cpg': cpg_df,
        'rewards': rewards_df,
        'original_metabrite': original_metabrite_df,
        'receipts': receipts_df,
        'receipt_items': receipt_items_df
    }


# Read the content from your JSON files
with open('receipts.json', 'r') as f:
    receipts_content = f.read()
with open('brands.json', 'r') as f:
    brands_content = f.read()
with open('users.json', 'r') as f:
    users_content = f.read()

# Process the data
tables = process_data(receipts_content, brands_content, users_content)

# Access all DataFrames
users_df = tables['users']
brands_df = tables['brands']
products_df = tables['products']
cpg_df = tables['cpg']
rewards_df = tables['rewards']
original_metabrite_df = tables['original_metabrite']
receipts_df = tables['receipts']
receipt_items_df = tables['receipt_items']



In [95]:
def analyze_table(df, table_name, unique_cols=None):
    """Perform all analyses on a single table and return a DataFrame with results."""
    analysis_data = []

    # Basic Summary
    analysis_data.append(["Total Records", len(df)])
    analysis_data.append(["Total Columns", df.shape[1]])
    analysis_data.append(["Columns", ', '.join(df.columns)])

    # Duplicate Analysis
    duplicate_rows = df[df.duplicated()]
    analysis_data.append(["Duplicate Rows", len(duplicate_rows)])

    if unique_cols:
        for col in unique_cols:
            if col in df.columns:
                dupes = df[df[col].duplicated(keep=False)]
                analysis_data.append([f"Duplicate {col}", len(dupes)])

    # Missing Values
    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0]
    for col, count in missing_values.items():
        analysis_data.append([f"Missing {col}", f"{count} ({round(count / len(df) * 100, 2)}%)"])

    # Data Types
    for col in df.columns:
        analysis_data.append([f"Data Type of {col}", str(df[col].dtype)])

    # Outlier Detection (IQR Method)
    for col in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
        analysis_data.append([f"Outliers in {col}", len(outliers)])

    return pd.DataFrame(analysis_data, columns=["Analysis", "Result"])


def check_relationships(tables):
    """Analyze foreign key relationships between tables and return a DataFrame."""
    relationships = [
        ('receipts', 'userId', 'users', '_id', "Receipt -> User"),
        ('receipt_items', 'receipt_id', 'receipts', '_id', "Receipt_Item_List -> Receipt"),
        ('receipt_items', 'barcode', 'products', 'barcode', "Receipt_Item_List -> Product"),
        ('receipt_items', 'originalMetaBrite_id', 'original_metabrite', '_id', "Receipt_Item_List -> OriginalMetaBrite"),
        ('receipt_items', 'pointsPayerId', 'rewards', 'pointsPayerId', "Receipt_Item_List -> Rewards"),
        ('products', 'brand_id', 'brands', '_id', "Product -> Brand"),
        ('products', 'cpg_id', 'cpg', '_id', "Product -> CPG")
    ]

    relationship_data = []
    for table_a, col_a, table_b, col_b, description in relationships:
        if table_a in tables and table_b in tables:
            total_rows = len(tables[table_a])
            matching_rows = tables[table_a][col_a].isin(tables[table_b][col_b]).sum()
            unmatched_rows = total_rows - matching_rows

            relationship_data.append([
                description, total_rows, matching_rows, unmatched_rows
            ])

    return pd.DataFrame(relationship_data, columns=["Relationship", "Total Records", "Matching Records", "Unmatched Records"])


# Example usage
tables = {
    'users': users_df,
    'brands': brands_df,
    'products': products_df,
    'cpg': cpg_df,
    'rewards': rewards_df,
    'original_metabrite': original_metabrite_df,
    'receipts': receipts_df,
    'receipt_items': receipt_items_df
}

# Save results to an Excel file
output_file = "Data_Analysis_Report.xlsx"
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for table_name, df in tables.items():
        analysis_df = analyze_table(df, table_name, unique_cols=['_id'])
        analysis_df.to_excel(writer, sheet_name=table_name[:31], index=False)

    # Add relationships as a separate sheet
    relationship_df = check_relationships(tables)
    relationship_df.to_excel(writer, sheet_name="Relationships", index=False)

print(f"Analysis report saved to {output_file}")


Analysis report saved to Data_Analysis_Report.xlsx
